In [14]:
from datetime import datetime as dt
from datetime import timedelta
from edgar import Company
import lxml.html as lh
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import re
import time

In [ ]:
# Functions for getting the CIK and Company Name from a SPAC ticker

def get_ticker_to_cik():
    # local copy: data/ticker_to_cik.txt
    ticker_to_cik = pd.read_csv('https://www.sec.gov/include/ticker.txt',
                                sep='\t', header=None, names=['ticker','cik'])
    ticker_to_cik['ticker'] = ticker_to_cik.ticker.str.upper()
    ticker_to_cik['cik'] = ticker_to_cik.cik.astype(str)
    return ticker_to_cik

def get_cik_to_name():
    # local copy: data/cik_to_name.json
    cik_to_name = pd.read_json('https://www.sec.gov/files/company_tickers.json').transpose()
    cik_to_name['ticker'] = cik_to_name.ticker.str.upper()
    cik_to_name['cik'] = cik_to_name.cik_str.astype(str)
    return cik_to_name

def process_spac_lists(file_path_current, file_path_past, write=False):
    # current spac list
    spac_list_current = pd.read_csv(file_path_current)
    spac_list_current = spac_list_current.Ticker.unique()
    spac_list_current = pd.DataFrame(spac_list_current, columns=['Ticker'])
    
    # past spac list (completed business combination)
    spac_list_past = pd.read_csv(file_path_past)
    spac_list_past.fillna('missing', inplace=True)
    spac_list_past['dupe_filter'] = spac_list_past['Old Ticker'] + spac_list_past['New Ticker']
    spac_list_past = spac_list_past[spac_list_past.dupe_filter.isin(spac_list_past.dupe_filter.unique())]
    spac_list_past.drop(columns=['dupe_filter'], inplace=True)
    
    # write to file
    if write==True:
        spac_list_current.to_csv('spac_list_current.csv', index=False)
        spac_list_past.to_csv('spac_list_past.csv', index=False)
    
    # get ticker to cik and cik to company name file, then merge
    ticker_to_cik = get_ticker_to_cik()
    cik_to_name = get_cik_to_name()
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='New Ticker', right_on='ticker')
    spac_list_past = spac_list_past.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    spac_list_current = spac_list_current.merge(ticker_to_cik, how='left', left_on='Ticker', right_on='ticker')
    spac_list_current = spac_list_current.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    
    return spac_list_current, spac_list_past

In [97]:
# Functions for pulling 8-K html, converting to text, grabbing date, and simple classifier

def form_html_to_text(forms_html):
    forms_text = []
    for form_html in forms_html:
        form_text = form_html.text_content().replace('\n',' ').replace('\xa0',' ').lower()
        form_text = re.sub(' +', ' ', form_text) # remove extra spaces
        forms_text.append(form_text)
    return forms_text

def date_parser_primary(split_text):
    # handle case: "november 13 (november 8, 2019)""
    if '(' in split_text[1] and ',' not in split_text[1].split('(')[0]:
        monthday_split = split_text[1].split('(')[0]
        year_split = split_text[1].split('(')[1].split(', ')[1].split(')')[0]
        date_string = monthday_split.replace(' ','') + ', ' + year_split.replace(')','')
    # handle cases:
    # "november 13, 2019"
    # "november 13, 2019 (november 15, 2019)"...uses first date
    # "july [   ], 2019"...return None
    else:
        split_text = split_text[1].split(', ')
        if '[' in split_text[0]:
            return None
        date_string = split_text[0].replace(' ','') + ', ' + split_text[1].replace(' ','')[0:4]
    date_dt = dt.strptime(date_string, '%B%d, %Y')
    date_dt = date_dt.strftime('%Y-%m-%d')
    return date_dt

def date_parser_secondary(split_text):
    # handle case when date comes before "date of report..." string
    split_text = split_text[0].split(', ')
    monthday_split = split_text[len(split_text)-2].split(' ')
    month = monthday_split[len(monthday_split)-2].replace('1934','').replace('(','') # case: 1934June
    day = monthday_split[len(monthday_split)-1]
    year = split_text[len(split_text)-1].replace(' ','')[0:4]
    date_string = str(month)+str(day)+', '+str(year)
    date_dt = dt.strptime(date_string, '%B%d, %Y')
    date_dt = date_dt.strftime('%Y-%m-%d')
    return date_dt

# TODO: handle duplicate dates (sometime multiple 8-Ks on same date)
def create_date_text_df(forms_text, form_type):
    df = pd.DataFrame()
    for form_text in forms_text:
        date_case_1 = 'date of report (date of earliest event reported):'
        date_case_2 = 'date of report (date earliest event reported):'
        date_case_3 = 'date of earliest event reported:'
        date_case_4 = 'date of report (date of earliest event reported)'
        if len(form_text.split(date_case_1))>1:
            split_text = form_text.split(date_case_1)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_2))>1:
            split_text = form_text.split(date_case_2)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_3))>1:
            split_text = form_text.split(date_case_3)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_4))>1:
            split_text = form_text.split(date_case_4)
            date_dt = date_parser_secondary(split_text)
        else:
            print('none of the date identifiers worked. skipping this form...')
#             print(form_text)
            continue
        df = df.append(pd.Series([date_dt, form_type, form_text]), ignore_index=True)
    if len(df)!=0:
        df.columns = ['date','form','text']
    return df

def get_forms_text(company_name, cik_id, form_type):
    print(company_name)
    company = Company(company_name, cik_id)
    print('url to forms:', company.get_filings_url(filing_type=form_type, ownership='include', no_of_entries=100))
    forms_site_html = company.get_all_filings(filing_type=form_type, ownership='include', no_of_entries=100)
    forms_html = company.get_documents(forms_site_html, no_of_documents=100, debug=False)
    forms_text = form_html_to_text(forms_html)
    if len(forms_text)==0:
        return
    df = create_date_text_df(forms_text, form_type)
    return df

def simple_text_match(df_form, substring):
    df_form[substring.replace(' ','_')+'_found'] = df_form.text.apply(lambda x: 1 if substring in x else 0)
    return df_form

In [129]:
# Functions for grabbing and saving price data

def bulk_save_alphavantage_data(symbols, start_date='2018-01-01', end_date='2020-07-10'):
    for symbol in symbols:
        print(symbol)
        df_prices = get_historical_prices(symbol=symbol,
                                          start_date=start_date,
                                          end_date=end_date,
                                          source='alphavantage')
        if df_prices is not None:
            df_prices = process_historical_prices(df_prices)
            df_prices.to_csv('data/prices/'+symbol+'_prices.csv', index=False)
        time.sleep(12)
        
def run_bulk_save_prices(spac_list_current, spac_list_past, warrants=False):
    # spacs missing price data entirely
    missing_past_spacs = ['missing', 'LCAH', 'FMCI1', 'CFCO']
    missing_current_spacs = ['ACNDU', 'ARYB', 'BRLI', 'DFHT', 'DMYD', 'FUSE', 'GOAC', 'IWAC', 'LCAH', 'LGVW',
                             'MCAC', 'MLAC', 'PANA', 'PSAC', 'PSTH', 'SSMC', 'TREB']
    
    symbols_past_new_ticker = [x for x in spac_list_past.ticker.unique().tolist() if str(x)!='nan']
    symbols_past_old_ticker = spac_list_past['Old Ticker'].unique().tolist()
    symbols_past_old_ticker = [x for x in symbols_past_old_ticker if x not in missing_past_spacs]
    symbols_current = [x for x in spac_list_current['Ticker'] if x not in missing_current_spacs]
    if warrants:
        symbols_past_new_ticker = [x + 'W' for x in symbols_past_new_ticker]
        symbols_past_old_ticker = [x + 'W' for x in symbols_past_old_ticker]
        symbols_current = [x + 'W' for x in symbols_current]
    bulk_save_alphavantage_data(symbols=symbols_past_new_ticker, start_date='2018-01-01', end_date='2020-07-10')
    bulk_save_alphavantage_data(symbols=symbols_past_old_ticker, start_date='2018-01-01', end_date='2020-07-10')
    bulk_save_alphavantage_data(symbols=symbols_current, start_date='2018-01-01', end_date='2020-07-10')
        
def load_saved_prices_data(symbol):
    df_prices = pd.read_csv('data/prices/'+symbol+'_prices.csv')
    print('price data min date:', df_prices.date.min())
    print('price data max date:', df_prices.date.max())
    return df_prices

def get_historical_prices(symbol, start_date, end_date, source):
    print('input start_date:', start_date)
    print('input end_date:', end_date)
    start_split = start_date.split('-')
    end_split = end_date.split('-')
    start = dt(int(start_split[0]), int(start_split[1]), int(start_split[2]))
    end = dt(int(end_split[0]), int(end_split[1]), int(end_split[2]))
    try:
        # be careful with missing/limited data in yahoo data
        if source=='yahoo':
            df_prices = pdr.data.DataReader(name=symbol, data_source='yahoo', start=start, end=end)
        # alphavantage seems to be most reliable. 5 calls per minute and 500 calls per day
        if source=='alphavantage':
            df_prices = pdr.data.DataReader(name=symbol, data_source='av-daily', start=start, end=end,
                                            api_key='BDB9WJQRCZKINCLD')
        # iex has extremely limited api calls
        if source=='iex':
            df_prices = pdr.data.DataReader(name=symbol, data_source='iex', start=start, end=end,
                                            api_key='pk_970dfff359894b15a056cf677c02e11f')
    except:
        return None
    return df_prices

def process_historical_prices(df_prices):
    df_prices.reset_index(inplace=True)
    df_prices.rename(columns={'index':'date'}, inplace=True)
    df_prices.columns = df_prices.columns.str.lower()
    df_prices['date'] = df_prices.date.astype(str)
    df_prices['close_t+1'] = df_prices.close.shift(-1)
    df_prices['close_t+3'] = df_prices.close.shift(-3)
    df_prices['close_t+5'] = df_prices.close.shift(-5)
    df_prices['close_t+7'] = df_prices.close.shift(-7)
    df_prices['open_close_t+1_%chg'] = (df_prices['close_t+1'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+3_%chg'] = (df_prices['close_t+3'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+5_%chg'] = (df_prices['close_t+5'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+7_%chg'] = (df_prices['close_t+7'] - df_prices['open']) / df_prices['open']
    df_prices = df_prices.round(2)
    print('output min date:', df_prices.date.min())
    print('output max date:', df_prices.date.max())
    return df_prices

In [121]:
# Function outputting dataframe of 8-Ks, classification, and 1,3,5,7 day returns

def agg_spac_returns(spac_list, price_source='alphavantage'):
    df_returns_agg = pd.DataFrame()
    for ind in range(0, len(spac_list)):
        broken_inds_current = [109,115,118,120,126]
        if ind in broken_inds_current:
            continue

        row = spac_list.iloc[ind]
        print('index:', ind)
        print(row.ticker)

        # get form 8Ks
        df_form_8K = get_forms_text(company_name=row.title, cik_id=row.cik, form_type='8-K')
        if df_form_8K is None or len(df_form_8K)==0:
            print('no 8Ks found\n')
            continue
            
        # simple classifier
        df_form_8K = simple_text_match(df_form_8K, 'letter of intent')
        df_form_8K = simple_text_match(df_form_8K, 'business combination agreement')

        # get stock prices
        try:
            df_prices = load_saved_prices_data(row.ticker)
        except:
            df_prices = get_historical_prices(symbol=row.ticker,
                                              start_date=df_form_8K.date.min(),
                                              end_date=(dt.strptime(df_form_8K.date.max(),'%Y-%m-%d') + 
                                                        timedelta(days=5)).strftime('%Y-%m-%d'),
                                              source=price_source)
            if df_prices is None:
                print('prices for', row.ticker, 'not found. skipping...')
                continue
            df_prices = process_historical_prices(df_prices)

        # output returns per form
        df_returns = df_form_8K.merge(df_prices[['date','open_close_t+1_%chg','open_close_t+3_%chg',
                                                 'open_close_t+5_%chg','open_close_t+7_%chg']],
                                      how='left', on='date')
        display(df_returns)

        # aggregate
        if len(df_returns)!=0:
            df_returns['symbol'] = row.ticker
        df_returns_agg = df_returns_agg.append(df_returns)
        
        if ind%20==0:
            time.sleep(60)
    return df_returns_agg

In [131]:
# load current and past spac lists
spac_list_current, spac_list_past = process_spac_lists(file_path_current='data/spac_list_current.csv',
                                                       file_path_past='data/spac_list_past.csv',
                                                       write=False)

# bulk save price data for symbols in spac lists (due to API limits)...no need to run if already saved
# run_bulk_save_prices(spac_list_current, spac_list_past, warrants=False
# run_bulk_save_prices(spac_list_current, spac_list_past, warrants=True)

In [86]:
# df_returns_past = agg_spac_returns(spac_list_past)
df_returns_current = agg_spac_returns(spac_list_current)

index: 0
ACAM
Acamar Partners Acquisition Corp.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1759008&type=8-K&dateb=&owner=include&count=100
price data min date: 2019-05-09
price data max date: 2020-07-10


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-02-20,8-K,8-k 1 tm209532d1_8k.htm form 8-k united state...,0,0,0.0,0.0,-0.0,-0.0
1,2019-04-12,8-K,8-k 1 tv518793_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
2,2019-04-10,8-K,8-k 1 tv518539_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
3,2019-03-04,8-K,8-k 1 tv515277_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
4,2019-02-26,8-K,8-k 1 tv514540_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN


index: 1
ACTT
Whole Earth Brands, Inc.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1753706&type=8-K&dateb=&owner=include&count=100
price data min date: 2019-05-14
price data max date: 2020-06-24


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-06-24,8-K,8-k12b/a 1 tm2023499-2_8k12ba.htm 8-k12b/a un...,0,0,NaN,NaN,NaN,NaN
1,2020-06-24,8-K,8-k12b 1 tm2023499d1_8k.htm form 8-k12b unite...,0,0,NaN,NaN,NaN,NaN
2,2020-06-15,8-K,8-k 1 tm2022707d1_8k.htm form 8-k united stat...,0,0,0.01,0.04,0.00,0.05
3,2020-06-15,8-K,8-k 1 tm2022614d1_8k.htm form 8-k united stat...,0,0,0.01,0.04,0.00,0.05
4,2020-06-08,8-K,8-k 1 tm2022016d1_8k.htm form 8-k united stat...,0,0,0.00,0.00,0.01,0.03
5,2020-05-08,8-K,8-k 1 tm2019220d1_8k.htm form 8-k united stat...,0,0,0.01,0.01,0.01,0.01
6,2020-03-27,8-K,8-k 1 ea120112actii_8k.htm united states secu...,0,0,0.01,0.00,-0.00,0.00
7,2020-02-12,8-K,8-k 1 tv538156_8k.htm 8-k united states secur...,0,0,0.02,0.01,0.01,0.01
8,2020-01-10,8-K,8-k 1 tv536223_8k.htm form 8-k united states ...,0,0,0.00,0.00,0.00,-0.00
9,2019-12-19,8-K,8-k 1 tv535070_8k.htm form 8-k united states ...,0,0,0.02,0.01,0.01,0.01


index: 2
ALAC
Alberton Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1748621&type=8-K&dateb=&owner=include&count=100
price data min date: 2018-11-21
price data max date: 2020-07-10


...

index: 123
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 124
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 125
NBAC
Newborn Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1780262&type=8-K&dateb=&owner=include&count=100
price data min date: 2020-05-01
price data max date: 2020-07-10


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-04-29,8-K,8-k 1 ea121166-8k_newbornacq.htm current repo...,0,0,NaN,NaN,NaN,NaN
1,2020-02-19,8-K,8-k 1 f8k021920b_newbornacq.htm current repor...,0,0,NaN,NaN,NaN,NaN
2,2020-02-13,8-K,8-k 1 f8k021320_newbornacq.htm current report...,0,0,NaN,NaN,NaN,NaN


index: 127
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 128
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 129
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 130
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 131
THBR
Thunder Bridge Acquisition II, LTD
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1769318&type=8-K&dateb=&owner=include&count=100
price data min date: 2019-10-04
price data max date: 2020-07-10


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2019-10-02,8-K,8-k 1 f8k100219_thunderbridge2.htm from 8-k u...,0,0,NaN,NaN,NaN,NaN
1,2019-08-19,8-K,8-k 1 f8k081319_thunderbridge2.htm current re...,0,0,NaN,NaN,NaN,NaN
2,2019-08-13,8-K,8-k 1 f8k080819_thunderbridge2.htm form 8k un...,0,0,NaN,NaN,NaN,NaN


index: 132
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 133
nan
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found



In [107]:
spac_list_current_warrants = spac_list_current.copy()
spac_list_current_warrants['ticker'] = spac_list_current_warrants.Ticker + 'W' # warrants
df_returns_current_warrants = agg_spac_returns(spac_list_current_warrants)

index: 0
ACAMW
Acamar Partners Acquisition Corp.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1759008&type=8-K&dateb=&owner=include&count=100
input start_date: 2019-02-26
input end_date: 2020-07-18
output min date: 2019-04-15
output max date: 2020-07-14


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-07-13,8-K,8-k 1 tm2024803d1_8k.htm form 8-k united stat...,0,0,-0.07,NaN,NaN,NaN
1,2020-02-20,8-K,8-k 1 tm209532d1_8k.htm form 8-k united state...,0,0,0.00,0.1,-0.11,-0.08
2,2019-04-12,8-K,8-k 1 tv518793_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
3,2019-04-10,8-K,8-k 1 tv518539_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
4,2019-03-04,8-K,8-k 1 tv515277_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
5,2019-02-26,8-K,8-k 1 tv514540_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN


index: 1
ACTTW
Whole Earth Brands, Inc.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1753706&type=8-K&dateb=&owner=include&count=100
input start_date: 2019-05-01
input end_date: 2020-06-29
output min date: 2019-05-14
output max date: 2020-06-24


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-06-24,8-K,8-k12b/a 1 tm2023499-2_8k12ba.htm 8-k12b/a un...,0,0,NaN,NaN,NaN,NaN
1,2020-06-24,8-K,8-k12b 1 tm2023499d1_8k.htm form 8-k12b unite...,0,0,NaN,NaN,NaN,NaN
2,2020-06-15,8-K,8-k 1 tm2022707d1_8k.htm form 8-k united stat...,0,0,0.12,0.54,0.07,0.27
3,2020-06-15,8-K,8-k 1 tm2022614d1_8k.htm form 8-k united stat...,0,0,0.12,0.54,0.07,0.27
4,2020-06-08,8-K,8-k 1 tm2022016d1_8k.htm form 8-k united stat...,0,0,0.06,-0.03,0.30,0.54
5,2020-05-08,8-K,8-k 1 tm2019220d1_8k.htm form 8-k united stat...,0,0,NaN,NaN,NaN,NaN
6,2020-03-27,8-K,8-k 1 ea120112actii_8k.htm united states secu...,0,0,NaN,NaN,NaN,NaN
7,2020-02-12,8-K,8-k 1 tv538156_8k.htm 8-k united states secur...,0,0,NaN,NaN,NaN,NaN
8,2020-01-10,8-K,8-k 1 tv536223_8k.htm form 8-k united states ...,0,0,0.04,0.21,0.21,0.31
9,2019-12-19,8-K,8-k 1 tv535070_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN


index: 2
ALACW
Alberton Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1748621&type=8-K&dateb=&owner=include&count=100
input start_date: 2018-10-23
input end_date: 2020-04-28
output min date: 2018-11-21
output max date: 2020-04-28


...

index: 122
LPROW
Open Lending Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1806201&type=8-K&dateb=&owner=include&count=100
input start_date: 2020-06-15
input end_date: 2020-06-20
prices for LPROW not found. skipping...
index: 123
MCACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 124
MLACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 125
NBACW
Newborn Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1780262&type=8-K&dateb=&owner=include&count=100
input start_date: 2020-02-13
input end_date: 2020-05-04
output min date: 2020-04-30
output max date: 2020-05-04


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-04-29,8-K,8-k 1 ea121166-8k_newbornacq.htm current repo...,0,0,NaN,NaN,NaN,NaN
1,2020-02-19,8-K,8-k 1 f8k021920b_newbornacq.htm current repor...,0,0,NaN,NaN,NaN,NaN
2,2020-02-13,8-K,8-k 1 f8k021320_newbornacq.htm current report...,0,0,NaN,NaN,NaN,NaN


index: 127
PANAW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 128
PSACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 129
PSTHW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 130
SSMCW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 131
THBRW
Thunder Bridge Acquisition II, LTD
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1769318&type=8-K&dateb=&owner=include&count=100
input start_date: 2019-08-13
input end_date: 2019-10-07
output min date: 2019-10-04
output max date: 2019-10-07


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2019-10-02,8-K,8-k 1 f8k100219_thunderbridge2.htm from 8-k u...,0,0,NaN,NaN,NaN,NaN
1,2019-08-19,8-K,8-k 1 f8k081319_thunderbridge2.htm current re...,0,0,NaN,NaN,NaN,NaN
2,2019-08-13,8-K,8-k 1 f8k080819_thunderbridge2.htm form 8k un...,0,0,NaN,NaN,NaN,NaN


index: 132
TREBW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 133
WPFW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found



Past SPACs

In [42]:
# bad price data: ACEL
corrupt_past = ['ACEL']

# note .123 is 12.3% in output
print('mean returns for past spacs')
df_mean_returns_past = df_returns_past[~df_returns_past.symbol.isin(corrupt_past)].groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'})
np.round(df_mean_returns_past.sort_values(by='open_close_t+3_%chg'), 3)

,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
GSMG,-0.058,-0.132,-0.073,-0.064
KXIN,-0.134,-0.109,-0.112,-0.127
AVCT,-0.052,-0.081,-0.098,-0.105
AESE,-0.033,-0.070,-0.079,-0.073
IMVT,0.008,-0.044,-0.072,-0.036
SPCE,-0.013,-0.044,-0.068,-0.063
BRMK,-0.017,-0.030,-0.044,-0.025
TH,-0.027,-0.027,-0.025,-0.015
HYMC,-0.000,-0.024,-0.038,-0.058


In [34]:
print('summary stats for returns in past spacs')
np.round(df_returns_past[~df_returns_past.symbol.isin(corrupt_past)].dropna().describe(), 3)

,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
count,832.000,832.000,832.000,832.000,832.000,832.000
mean,0.012,0.054,-0.005,0.005,-0.006,-0.010
std,0.109,0.226,0.119,0.313,0.223,0.217
min,0.000,0.000,-0.661,-0.810,-0.820,-0.840
25%,0.000,0.000,-0.030,-0.050,-0.060,-0.073
50%,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,0.000,0.020,0.020,0.020,0.020
max,1.000,1.000,1.140,6.320,3.390,2.380


In [45]:
print('mean returns for past spacs (LOI text match)')
np.round(df_returns_past[(~df_returns_past.symbol.isin(corrupt_past)) &
                         (df_returns_past.letter_of_intent_found==1)].dropna().groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'}), 3)

,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
AVCT,0.000,0.0,0.000,0.000
KLR,-0.002,0.0,0.006,-0.001
XYNO,0.000,0.0,0.000,0.000


Current SPACs

In [92]:
# bad price data
corrupt_current = []

# note .123 is 12.3% in output
print('mean returns for current spacs')
df_mean_returns_current = df_returns_current[~df_returns_current.symbol.isin(corrupt_current)].groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'})
np.round(df_mean_returns_current.dropna().sort_values(by='open_close_t+3_%chg'), 3)

mean returns for current spacs


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
LCA,-0.103,-0.122,-0.160,-0.215
INSU,-0.065,-0.080,-0.080,-0.095
CRSA,-0.055,-0.060,-0.068,-0.068
TZAC,-0.043,-0.040,-0.043,-0.043
MUDS,-0.000,-0.024,-0.038,-0.058
LIVK,0.000,-0.020,-0.030,-0.070
GRAF,0.027,-0.017,0.033,0.000
OPES,-0.000,-0.014,-0.009,0.022
HECCU,-0.010,-0.010,-0.010,-0.010


In [95]:
print('summary stats for returns in current spacs')
np.round(df_returns_current[~df_returns_current.symbol.isin(corrupt_current)].dropna().describe(), 3)

summary stats for returns in current spacs


,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
count,385.000,385.000,385.000,385.000,385.000,385.000
mean,0.036,0.104,0.001,0.008,0.006,0.011
std,0.187,0.306,0.045,0.061,0.066,0.129
min,0.000,0.000,-0.280,-0.220,-0.290,-0.310
25%,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.000,0.000,0.000,0.000,-0.000
75%,0.000,0.000,0.000,0.010,0.010,0.010
max,1.000,1.000,0.380,0.710,0.710,2.120


In [96]:
print('mean returns for current spacs (LOI text match)')
np.round(df_returns_current[(~df_returns_current.symbol.isin(corrupt_current)) &
                            (df_returns_current.letter_of_intent_found==1)].dropna().groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'}), 3)

mean returns for current spacs (LOI text match)


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
ALAC,-0.003,0.000,0.000,0.007
AMCI,0.000,0.000,0.000,0.010
BMRG,0.110,0.130,0.100,0.120
FMCI,0.050,0.125,0.155,0.080
JEF,-0.030,0.090,0.090,0.100
MFAC,0.000,0.010,0.020,0.050
NFIN,-0.020,-0.010,-0.020,-0.010
OPES,0.120,0.110,0.200,0.580
PAAC,0.000,0.010,0.000,0.000


Current SPAC Warrants

In [108]:
# bad price data
corrupt_current = []

# note .123 is 12.3% in output
print('mean returns for current spacs')
df_mean_returns_current = df_returns_current_warrants[~df_returns_current_warrants.symbol.isin(corrupt_current)].groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'})
np.round(df_mean_returns_current.dropna().sort_values(by='open_close_t+3_%chg'), 3)

mean returns for current spacs


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
LOACW,-0.060,-0.060,-0.194,-0.128
THCBW,0.077,-0.005,0.075,0.045
TDACW,0.220,-0.003,0.175,0.100
LACQW,0.060,0.030,-0.033,-0.063
ALACW,0.111,0.040,0.130,-0.032
HCCHW,0.072,0.048,0.014,0.086
LFACW,-0.010,0.050,-0.100,-0.050
NEBUW,0.062,0.099,0.133,0.129
ACAMW,-0.035,0.100,-0.110,-0.080


In [109]:
print('summary stats for returns in current spacs')
np.round(df_returns_current_warrants[~df_returns_current_warrants.symbol.isin(corrupt_current)].dropna().describe(), 3)

summary stats for returns in current spacs


,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
count,93.000,93.000,93.000,93.000,93.000,93.000
mean,0.054,0.118,0.134,0.212,0.245,0.311
std,0.227,0.325,0.538,0.649,0.856,1.069
min,0.000,0.000,-0.380,-0.380,-0.650,-0.650
25%,0.000,0.000,-0.050,-0.050,-0.090,-0.110
50%,0.000,0.000,0.000,0.030,0.070,0.060
75%,0.000,0.000,0.120,0.250,0.270,0.250
max,1.000,1.000,3.520,3.760,4.800,7.000


In [110]:
print('mean returns for current spacs (LOI text match)')
np.round(df_returns_current_warrants[(~df_returns_current_warrants.symbol.isin(corrupt_current)) &
                            (df_returns_current_warrants.letter_of_intent_found==1)].dropna().groupby('symbol').agg(
    {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
     'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'}), 3)

mean returns for current spacs (LOI text match)


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
ALACW,0.10,0.10,0.200,0.100
AMCIW,-0.18,0.65,0.650,0.880
FMCIW,1.82,2.12,2.795,2.495
OPESW,3.18,3.13,4.180,7.000
